# Cleaning
---

In [1]:
import pandas as pd
import numpy as np

## Solar Generation Data

In [3]:
df = pd.read_csv('Solar_Energy_Generation.csv')
df.head()

,CampusKey,SiteKey,Timestamp,SolarGeneration
0,2,1,2020-01-01 00:15:00,NaN
1,2,1,2020-01-01 00:30:00,NaN
2,2,1,2020-01-01 00:45:00,NaN
3,2,1,2020-01-01 01:00:00,NaN
4,2,1,2020-01-01 01:15:00,NaN


In [4]:
df.shape

(2731946, 4)

Missing Data:

In [5]:
#Solar Generation at night is reported as NaN
df.isnull().sum()

CampusKey                0
SiteKey                  0
Timestamp                0
SolarGeneration    1536301
dtype: int64

In [6]:
#Checking to see if any missing data is during the day
df_na = df[df.isnull()]
df_na['Timestamp'] = pd.to_datetime(df_na['Timestamp'])
df_na['time'] = df_na.Timestamp.dt.time
df_na[(df_na['time']> '09:00:00') & (df_na['time'] < '17:00:00')]

,CampusKey,SiteKey,Timestamp,SolarGeneration,time


In [7]:
#Filling Solar Generation at night equal to 0
df.fillna(0, inplace = True)

Setting Index as Datetime and Combining All Campuses Into Combined Generation Across All Sites

In [8]:
df1 = pd.DataFrame(df.groupby(['Timestamp', 'CampusKey'])['SolarGeneration'].sum())
df1.reset_index(inplace = True)
df1['Timestamp'] = pd.to_datetime(df1['Timestamp'])
df1['time'] = df1.Timestamp.dt.time

In [9]:
df1.set_index('Timestamp', inplace = True, drop = True)

In [10]:
camp1 = df1[df1['CampusKey'] == 1]
camp2 = df1[df1['CampusKey'] == 2]
camp3 = df1[df1['CampusKey'] == 3]
camp4 = df1[df1['CampusKey'] == 4]
camp5 = df1[df1['CampusKey'] == 5]

---

## Weather Data

In [11]:
weather = pd.read_csv('Weather_Data_reordered_all.csv')
weather.head()

,CampusKey,Timestamp,ApparentTemperature,AirTemperature,DewPointTemperature,RelativeHumidity,WindSpeed,WindDirection
0,1,2020-01-01 00:00:00,13.666667,13.880000,8.960000,72.400000,0.000000,188.133333
1,1,2020-01-01 00:15:00,13.206667,13.666667,9.040000,73.466667,1.200000,203.866667
2,1,2020-01-01 00:30:00,12.840000,13.553333,9.053333,74.000000,2.520000,222.800000
3,1,2020-01-01 00:45:00,12.113333,13.506667,9.100000,74.466667,5.986667,231.133333
4,1,2020-01-01 01:00:00,11.946667,13.260000,9.266667,76.533333,5.946667,247.866667


In [12]:
weather.CampusKey.unique()

array([1, 2, 3, 4, 5])

Setting Index as Datetime and Combining All Campuses Into Combined Weather Data Across All Sites

In [13]:
weather['Timestamp'] = pd.to_datetime(weather['Timestamp'])
weather.set_index('Timestamp', inplace = True)
weather.sort_index(inplace = True)

In [14]:
weather.shape

(371769, 7)

In [15]:
weather.isnull().sum()

CampusKey                   0
ApparentTemperature    107113
AirTemperature         107113
DewPointTemperature    107113
RelativeHumidity       107113
WindSpeed              162890
WindDirection          162890
dtype: int64

In [16]:
campus1 = weather[weather['CampusKey'] == 1]
campus2 = weather[weather['CampusKey'] == 2]
campus3 = weather[weather['CampusKey'] == 3]
campus4 = weather[weather['CampusKey'] == 4]
campus5 = weather[weather['CampusKey'] == 5]

In [17]:
campus5.head()

,CampusKey,ApparentTemperature,AirTemperature,DewPointTemperature,RelativeHumidity,WindSpeed,WindDirection
Timestamp,,,,,,,
2020-12-13 01:00:00,5,15.173333,19.386667,7.786667,47.000000,19.04,36.533333
2020-12-13 01:15:00,5,15.113333,19.186667,7.893333,48.000000,18.38,33.933333
2020-12-13 01:30:00,5,14.680000,19.166667,7.940000,48.200000,20.60,34.866667
2020-12-13 01:45:00,5,14.640000,19.146667,7.846667,48.000000,20.64,35.400000
2020-12-13 02:00:00,5,14.966667,18.906667,7.846667,48.533333,17.64,39.733333


## Merging Solar Generation Data with Weather Data Per Campus

In [68]:
camp1.shape, campus1.shape

((79805, 3), (81017, 7))

In [20]:
df1 = pd.merge(left = camp1, right = campus1, on=['Timestamp', 'CampusKey'], how='left')
df1.to_csv('campus1.csv')
df2 = pd.merge(left = camp2, right = campus2, on=['Timestamp', 'CampusKey'], how='left')
df2.to_csv('campus2.csv')
df3 = pd.merge(left = camp3, right = campus3, on=['Timestamp', 'CampusKey'], how='left')
df3.to_csv('campus3.csv')
df4 = pd.merge(left = camp4, right = campus4, on=['Timestamp', 'CampusKey'], how='left')
df4.to_csv('campus4.csv')
df5 = pd.merge(left = camp5, right = campus5, on=['Timestamp', 'CampusKey'], how='left')
df5.to_csv('campus5.csv')

In [21]:
df['SiteKey'][df['CampusKey'] == 1].unique()

array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31, 32, 33, 34, 35, 36, 37, 38, 39, 40])

In [22]:
df['SiteKey'][df['CampusKey'] == 2].unique()

array([1, 2, 3, 4, 5])

In [23]:
df['SiteKey'][df['CampusKey'] == 3].unique()

array([ 6,  7,  8,  9, 10, 11, 12, 13])

In [24]:
df['SiteKey'][df['CampusKey'] == 4].unique()

array([41])

In [25]:
df['SiteKey'][df['CampusKey'] == 5].unique()

array([42])

### Exporting Half of the Individual Sites on Campus 3 For Granular Investigation

In [28]:
df_camp3 = df[df['CampusKey'] == 3].copy()
df_camp3['Timestamp'] = pd.to_datetime(df_camp3['Timestamp'])
df_camp3['time'] = df_camp3.Timestamp.dt.time

df_camp3_site6 = df_camp3[df_camp3['SiteKey'] == 6]
df_camp3_site6.to_csv('campus3_site6.csv')

df_camp3_site8 = df_camp3[df_camp3['SiteKey'] == 8]
df_camp3_site8.to_csv('campus3_site8.csv')

df_camp3_site10 = df_camp3[df_camp3['SiteKey'] == 10]
df_camp3_site10.to_csv('campus3_site10.csv')

df_camp3_site12 = df_camp3[df_camp3['SiteKey'] == 12]
df_camp3_site12.to_csv('campus3_site12.csv')